# Сегментация тетрадей

В данном ноутбуке представлено обучение модели instance сегментации текста в школьных тетрадях с помощью detectron2. Мыиспользовали аугментации и модель R101-FPN.

# Установка библиотек

Установка библиотек, под которым запускается данный бейзлайн.

In [ ]:
!pip install pyyaml==5.1

import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/$CUDA_VERSION/torch$TORCH_VERSION/index.html

# Загружаем необходимые библиотеки для создания и обучения модели

In [ ]:
import cv2
import random
import json
import os

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import shutil

In [ ]:
import tqdm

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
import numpy as np

In [ ]:
import torch, torchvision
import detectron2
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog,DatasetCatalog
from detectron2.data.datasets import register_coco_instances,load_coco_json
from detectron2.data import detection_utils as utils
from detectron2.engine import DefaultTrainer
from detectron2.engine import HookBase



import logging
logger = logging.getLogger('detectron2')
logger.setLevel(logging.CRITICAL)

Прежде чем переходить к загрузке данных посмотрим, доступны ли нам GPU-мощности. 

In [ ]:
print('GPU: ' + str(torch.cuda.is_available()))

GPU: True


# Валидационный датасет

In [ ]:
!wget https://storage.yandexcloud.net/datasouls-competitions/ai-nto-final-2022/data.zip

In [ ]:
!unzip data.zip

Для валидации наших моделей нам неплохо было создать из обучающих данных валидационный датасет. Для этого разделим наш датасет на две части - для обучения и для валидации. Для этого просто создадим два новых файлика с аннотациями, куда раздельно запишем исиходную информацию об аннотациях.

# Регистрация датасета

Зарегистрируем выборки в detectron2 для дальнейшей подачи на обучение модели.

In [ ]:
for d in ['train','val']:
    DatasetCatalog.register("my_dataset2_"+d, lambda d=d: load_coco_json("data/train_segmentation/annotations.json",
    image_root= "data/train_segmentation/images",\
    dataset_name="my_dataset2_"+d,extra_annotation_keys=['bbox_mode']))

После регистрации можно загружать выборки, чтобы иметь возможность посмотреть на них глазами. Первой загрузим обучающую выборку в **dataset_dicts_train**

In [ ]:
dataset_dicts_train = DatasetCatalog.get("my_dataset2_train")
train_metadata = MetadataCatalog.get("my_dataset2_train")

И тестовую выборку в **dataset_dicts_val**

In [ ]:
dataset_dicts_val = DatasetCatalog.get("my_dataset2_val")
val_metadata = MetadataCatalog.get("my_dataset2_val")

Посмотрим на размер получившихся выборок - эта операция в python осуществляется при помощи функции **len()**

In [ ]:
print('Размер обучающей выборки (Картинки): {}'.format(len(dataset_dicts_train)))
print('Размер тестовой выборки (Картинки): {}'.format(len(dataset_dicts_val)))

Размер обучающей выборки (Картинки): 932
Размер тестовой выборки (Картинки): 932


Итак, у нас в распоряжении 588 изображения для тренировки, и 66 - для проверки качества.

**Посмотрим на размеченные фотографии из валидации**

In [ ]:
import os
from IPython.display import Image
@interact
def show_images(file=range(len(dataset_dicts_val))):
    example = dataset_dicts_val[file]
    image = utils.read_image(example["file_name"], format="RGB")
    plt.figure(figsize=(3,3),dpi=200)
    visualizer = Visualizer(image[:, :, ::-1], metadata=val_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(example)
    plt.imshow(vis.get_image()[:, :,::-1])
    plt.show()

## Обучение модели

**4.1. Определяем конфигурацию**

Прежде чем начать работать с самой моделью, нам нужно определить ее параметры и спецификацию обучения

Создаем конфигурацию и загружаем архитектуру модели с предобученными весами (на COCO - датасете, содержащем $80$ популярных категорий объектов и более $300000$ изображений) для распознавания объектов.

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml")) 
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml")

В целом, вы можете посмотреть и другие архитектуры в зоопарке [моделей](https://github.com/facebookresearch/detectron2/blob/master/MODEL_ZOO.md).

Теперь задаем параметры самой модели и обучения модели

In [ ]:
# Загружаем названия обучающией и тестовой выборок в настройки
cfg.DATASETS.TRAIN = ("my_dataset2_train",)
cfg.DATASETS.TEST = ("my_dataset2_val",)
cfg.TEST.EVAL_PERIOD = 500000
cfg.TEST.DETECTIONS_PER_IMAGE = 1000
cfg.INPUT.MIN_SIZE_TEST= 1960
cfg.INPUT.MAX_SIZE_TEST = 2016
# Часто имеет смысл сделать изображения чуть меньшего размера, чтобы 
# обучение происходило быстрее. Поэтому мы можем указать размер, до которого будем изменяться наименьшая 
# и наибольшая из сторон исходного изображения.
cfg.INPUT.MIN_SIZE_TRAIN = 1960
cfg.INPUT.MAX_SIZE_TRAIN = 2016

# Также мы должны сказать модели ниже какой вероятности определения она игнорирует результат. 
# То есть, если она найдет на картинке еду, но вероятность правильного определения ниже 0.5, 
# то она не будет нам сообщать, что она что-то нашла.
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5

# Также мы должны указать порядок каналов во входном изображении. Обратите внимание, что это Blue Green Red (BGR), 
# а не привычный RGB. Это особенности работы данной модели.
cfg.INPUT.FORMAT = 'BGR' 

# Для более быстрой загрузки данных в модель, мы делаем параллельную загрузку. Мы указываем параметр 4, 
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 1024
cfg.MODEL.RPN.BATCH_SIZE_PER_IMAGE = 256

# Следующий параметр задает количество изображений в батче, на котором 
# модель делает одну итерацию обучения (изменения весов).
cfg.SOLVER.IMS_PER_BATCH = 2

# Зададим также learning_rate
cfg.SOLVER.BASE_LR = 0.01

# Укажем модели, через сколько шагов обучения модели следует уменьшить learning rate
cfg.SOLVER.STEPS = (24000,38000,50000)

# Фактор, на который уменьшается learning rate задается следующим выражением
cfg.SOLVER.GAMMA = 0.5
#cfg.SOLVER.WEIGHT_DECAY = 0.01

# Зададим общее число итераций обучения.
cfg.SOLVER.MAX_ITER = 58000

# Укажем количество классов в нашей выборке
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1

# Задаем через сколько  шагов обучения сохранять веса модели в файл. Этот файл мы сможем загрузить потом 
# для тестирования нашей обученной модели на новых данных.
cfg.SOLVER.CHECKPOINT_PERIOD = 2000

# И указываем название папки, куда сохранять чекпойнты модели и информацию о процессе обучения.
cfg.OUTPUT_DIR = 'outputs'

# Если вдруг такой папки нет, то создадим ее
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

# Если мы хотим удалить чекпойнты предыдущих моделей, то выполняем данную команду. 
#%rm output/*

**4.2. Обучаем модель**

Процесс обучения модели запускают следующие три строчки кода. Возможно будут предупреждения, на которые можно не обращать внимания, это информация об обучении.

In [ ]:
!pip install albumentations==1.0.0
!pip uninstall opencv-python-headless==4.5.5.62 -y
!pip install opencv-python-headless==4.1.2.30

     |████████████████████████████████| 98 kB 2.0 MB/s 
     |████████████████████████████████| 47.7 MB 112 kB/s 
  Attempting uninstall: albumentations
    Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


Found existing installation: opencv-python-headless 4.5.5.62
Uninstalling opencv-python-headless-4.5.5.62:
  Successfully uninstalled opencv-python-headless-4.5.5.62
     |████████████████████████████████| 21.8 MB 179 kB/s 


In [ ]:
import albumentations as A
a_transforms = A.Compose([
                        A.OneOf([
                             A.RGBShift(p=1),
                             A.HueSaturationValue(p=1),
                             A.CLAHE(p=1),
                        ], p=0.2),
                        A.OneOf([
                             A.Blur(blur_limit=7,p=1),
                             A.GaussianBlur(p=1),
                             A.MedianBlur (blur_limit=7,p=1),
                        ], p=0.2),
                        A.OneOf([
                             A.RandomBrightnessContrast(brightness_limit=0.5, contrast_limit=0.5, p=1),
                             A.RandomGamma (gamma_limit=(80, 150), p=1),
                             A.RandomToneCurve(p=1),
                        ], p=0.2),
                        A.OneOf([
                             A.ColorJitter(p=1),
                             A.JpegCompression(p=1),
                             A.GaussNoise(p=1),
                        ], p=0.2),
                        A.RandomShadow (p=0.2),
    ])

In [ ]:
import detectron2.data.transforms as T
import copy
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader, build_detection_train_loader
def custom_mapper(dataset_dict):
    # Implement a mapper, similar to the default DatasetMapper, but with your own customizations
    dataset_dict = copy.deepcopy(dataset_dict)  # it will be modified by code below
    image = utils.read_image(dataset_dict["file_name"], format="BGR")
    transform_list = [T.ResizeShortestEdge([1960, 1960], 2016),
                      T.RandomFlip(prob=0.5, horizontal=False, vertical=True),
                      T.RandomFlip(prob=0.5, horizontal=True, vertical=False), 
                      T.RandomRotation(angle=[-45, 45]),
                      T.RandomCrop(crop_type='relative_range', crop_size=[0.8, 0.8])
                      ]
    image, transforms = T.apply_transform_gens(transform_list, image)
    image = a_transforms(image=image)['image']
    dataset_dict["image"] = torch.as_tensor(image.transpose(2, 0, 1).astype("float32"))

    annos = [
        utils.transform_instance_annotations(obj, transforms, image.shape[:2])
        for obj in dataset_dict.pop("annotations")
        if obj.get("iscrowd", 0) == 0
    ]
    instances = utils.annotations_to_instances(annos, image.shape[:2])
    dataset_dict["instances"] = utils.filter_empty_instances(instances)
    return dataset_dict


class CustomTrainer(DefaultTrainer):
    
    @classmethod
    def build_train_loader(cls, cfg):
        return build_detection_train_loader(cfg, mapper=custom_mapper)

In [ ]:
trainer = CustomTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

Используем обученную модель для проверки качества на валидации.

In [ ]:
cfg.MODEL.WEIGHTS = "outputs/model_final.pth"

cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.3
cfg.DATASETS.TEST = ("my_dataset_val", )
#Изменение размера исходных изображений для тестового датасета
cfg.INPUT.MIN_SIZE_TEST= 1960
cfg.INPUT.MAX_SIZE_TEST = 2016
cfg.INPUT.FORMAT = 'BGR'

#ВАЖНО увеличить это значение (стандартное равно 100). Так как на листе тетради может быть довольно много слов
cfg.TEST.DETECTIONS_PER_IMAGE = 1000

predictor = DefaultPredictor(cfg)

Сделаем предсказания для тестового датасета и сразу же нарисуем его.

Вы можете выбрать из выпадающего списка номер изображения, и посмотреть разметку на всем валидационном датасете.

In [ ]:
@interact
def show_images(file=range(len(dataset_dicts_val))):
    
    example = dataset_dicts_val[file]
    im = cv2.imread(example["file_name"])
    outputs = predictor(im)
    fig, axs = plt.subplots(nrows=1,ncols=2,figsize=(4,4),dpi=200)
    v = Visualizer(im[:, :],
                  metadata=val_metadata, 
                  scale=0.4 )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    axs[0].imshow(im[:, :, ::-1])
    axs[1].imshow(v.get_image()[:, :, ::-1])
    axs[0].axis('off')
    axs[1].axis('off')
    axs[0].set_title('Original')
    axs[1].set_title('Predict')
    plt.show()

Можно непосредственно в коде изменить номер изображения, которое Вы хотите обработать.

In [ ]:
id_image_selected = 3
example = dataset_dicts_val[id_image_selected]
im = cv2.imread(example["file_name"])
outputs = predictor(im)
plt.figure(figsize=(7,7))
v = Visualizer(im[:, :],
              metadata=val_metadata, 
              scale=0.4 )
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
plt.imshow(v.get_image()[:, :, ::-1])
plt.axis('off')
plt.show()

In [ ]:
outputs['instances']

В качестве предсказаний для каждого изображения из тестового набора требуется получить бинарную маску, в которой `1` означает, что данный пиксель относится к классу текста.

Давайте на примере одного изображения переведем формат выхода Detectron2 в требуемый формат для соревнования.

`outputs` - результат предсказания модели на данном изображении из предыдущего блока с кодом

In [ ]:
prediction = outputs['instances'].pred_masks.cpu().numpy()

In [ ]:
prediction.shape

В `prediction` находится массив бинарных матриц. Каждая матрица отвечает за отдельную задетектированную маску текста. В нашем случае модель задетектировала 80 текстовых масок. Давайте провизуализируем одну из них.

In [ ]:
prediction[0]

In [ ]:
plt.imshow(prediction[0])

Отлично. Теперь, для того, чтобы получить бинарную маску со всем задетектированным текстом для изображения, нам нужно объединить все маски в одну. Для этого мы просто поэелементно сложим все наши матрицы. Там, где после сложения остались нули - модель не задетектировала никакого текста.

In [ ]:
mask = np.add.reduce(prediction)

In [ ]:
mask = mask > 0

In [ ]:
plt.imshow(mask)

Итак, нам нужно полуить такую маску для каждого изображения из валидационной выборки, а затем посчитать метрику F1-score.

In [ ]:
#Подгрузим аннотации train
with open('data/val/annotations_new.json') as f:
    annotations_val = json.load(f)

In [ ]:
val_images = annotations_val['images']

In [ ]:
val_predictions = {}

In [ ]:
with torch.no_grad():
  for val_img in tqdm.tqdm_notebook(val_images):
      file_name = val_img['file_name']
      img_path = os.path.join('data/val/images/',file_name)
      im = cv2.imread(img_path)
      outputs = predictor(im)
      prediction = outputs['instances'].pred_masks.cpu().numpy()
      mask = np.add.reduce(prediction)
      mask = mask > 0
      val_predictions[file_name] = mask

Для сохрания предсказаний и загрузки бинарных масок бы будет использовать формат `.npz`. Он позволяет хранить большие массивы в компактном виде. Вот [ссылка](https://numpy.org/doc/stable/reference/generated/numpy.savez_compressed.html) на документацию.

In [ ]:
np.savez_compressed('val_pred.npz',**val_predictions)

Подгрузим бинарные маски для train и val (только что сохраненную). Так как мы в начале бейзлайна разбивали весь исходный train на новый трейн и валидацию, то информация по всем маскам из исходного train хранится в `binary.npz`. 

Получившийся после подгрузки `np.load()` - что то вроде словаря. Его ключи можно получить с помощью метода files - `loaded_val.files`. В нашем случае ключами являются ключи исходного словаря `val_predictions`, то есть названия изображений.

In [ ]:
loaded_train = np.load('train_data/binary.npz')

In [ ]:
loaded_val_pred = np.load('val_pred.npz')

Мы используем среднюю метрика F1-score. То есть считаем F1-score для каждого изображения, а затем усредняем результаты. 

Реализация из sklearn работает довольно долго, попэтому мы будем использовать свою.

In [ ]:
def f1_loss(y_true, y_pred):
        
    
    tp = np.sum(y_true & y_pred)
    tn = np.sum(~y_true & ~y_pred)
    fp = np.sum(~y_true & y_pred)
    fn = np.sum(y_true & ~y_pred)
    
    epsilon = 1e-7
    
    precision = tp / (tp + fp + epsilon)
    recall = tp / (tp + fn + epsilon)
    
    f1 = 2* precision*recall / ( precision + recall + epsilon)

    return f1 

In [ ]:
f1_scores = []
for key in tqdm.tqdm_notebook(loaded_val_pred.files):
    pred = loaded_val_pred[key].reshape(-1)
    true = loaded_train[key].reshape(-1)
    
    f1_img = f1_loss(true,pred)
    f1_scores.append(f1_img)

Получившаяся метрика на валидации.

In [ ]:
np.mean(f1_scores)